In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
import pandas as pd
from pydantic import BaseModel, Field
from typing import List

In [2]:
loaders = [PyPDFLoader('data/fattura.pdf')]

docs = []
for file in loaders:
    docs.extend(file.load())

docs[0]

Document(metadata={'producer': 'GPL Ghostscript 10.05.1', 'creator': 'InvoiceHome.com', 'creationdate': "D:20250723100914Z00'00'", 'moddate': "D:20250723100914Z00'00'", 'author': 'InvoiceHome.com', 'source': 'data/fattura.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Azienda srl\nBill To Invoice #\n42\nAzienda SpA Invoice Date 23/07/2025\nDescription Amount\n1 x DLRG-23 Faretti LED 40W RGB 118.50\n1 x STRD-213 Asta microfonica Rode PSA1+ per microfoni a condensatore 104. 99\n1 x MCRD-12 Microfono Rode PodMic-USB 139.00\n2 x CVSB-71 Cavo USB-C / USB-A 3 metri schermato 9.99\n1 x WBLG-9 Webcam Logitech Brio 4k con supporto da monitor 129.91\n1 x STDK-3 Elgato Stream Deck 15 tasti 119.99\nSubtotal 622.38\ni.v.a. 22.0% 136.93\nInvoice Total 759.31 €\nTerms & Conditions\nIl pagamento deve essere e9ettuato entro 15 giorni')

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("openai_key"))

In [4]:
print(llm.invoke("elenca articoli e quantità presenti nella seguente fattura:\n" + docs[0].page_content).content)

Ecco l'elenco degli articoli e le relative quantità presenti nella fattura:

1. **DLRG-23 Faretti LED 40W RGB** - 1 pezzo - 118.50 €
2. **STRD-213 Asta microfonica Rode PSA1+ per microfoni a condensatore** - 1 pezzo - 104.99 €
3. **MCRD-12 Microfono Rode PodMic-USB** - 1 pezzo - 139.00 €
4. **CVSB-71 Cavo USB-C / USB-A 3 metri schermato** - 2 pezzi - 9.99 € ciascuno
5. **WBLG-9 Webcam Logitech Brio 4k con supporto da monitor** - 1 pezzo - 129.91 €
6. **STDK-3 Elgato Stream Deck 15 tasti** - 1 pezzo - 119.99 €

Queste informazioni riassumono gli articoli e le quantità della fattura fornita.


In [5]:
class Product(BaseModel):
    """details of an item or product present in a document"""

    product_code: str = Field(default="", description="the product code associated with an item")
    description: str = Field(default="", description="the description associated with an item")
    quantity: str = Field(default="", description="how many items are in an order, can have different units of measurement")
    price: float = Field(default="", description="the price of the item")


class Products_List(BaseModel):
    """Identifying information about all items in a document"""

    products: List[Product]

In [6]:
parser = PydanticOutputParser(pydantic_object=Products_List)

prompt = PromptTemplate(
    template="{query}\n\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

parsed_chain = prompt | llm | parser

In [9]:
articoli_fattura = parsed_chain.invoke(
    "elenca codice articoli, descrizione articoli, quantità e prezzo presenti nella seguente fattura:\n" + docs[
        0].page_content)

df = pd.DataFrame([{
    "Codice Prodotto": p.product_code,
    "Descrizione": p.description,
    "Quantità": p.quantity,
    "Prezzo": p.price
} for p in articoli_fattura.products])

In [10]:
df

,Codice Prodotto,Descrizione,Quantità,Prezzo
0,DLRG-23,Faretti LED 40W RGB,1,118.50
1,STRD-213,Asta microfonica Rode PSA1+ per microfoni a co...,1,104.99
2,MCRD-12,Microfono Rode PodMic-USB,1,139.00
3,CVSB-71,Cavo USB-C / USB-A 3 metri schermato,2,9.99
4,WBLG-9,Webcam Logitech Brio 4k con supporto da monitor,1,129.91
5,STDK-3,Elgato Stream Deck 15 tasti,1,119.99


In alternativa a quanto fatto tramite il parser di pydantic, possiamo anche utilizzate il metodo with_structured_output
che forza il modello a rispondere in un formato strutturato JSON attraverso lo schema pydantic specificato

In [17]:
products_llm = llm.with_structured_output(schema=Products_List)

In [18]:
articoli_fattura = products_llm.invoke("elenca codice articoli, descrizione articoli, quantità e prezzo presenti nella seguente fattura:\n" + docs[0].page_content)

articoli_fattura

Products_List(products=[Product(product_code='DLRG-23', description='Faretti LED 40W RGB', quantity='1', price=118.5), Product(product_code='STRD-213', description='Asta microfonica Rode PSA1+ per microfoni a condensatore', quantity='1', price=104.99), Product(product_code='MCRD-12', description='Microfono Rode PodMic-USB', quantity='1', price=139.0), Product(product_code='CVSB-71', description='Cavo USB-C / USB-A 3 metri schermato', quantity='2', price=9.99), Product(product_code='WBLG-9', description='Webcam Logitech Brio 4k con supporto da monitor', quantity='1', price=129.91), Product(product_code='STDK-3', description='Elgato Stream Deck 15 tasti', quantity='1', price=119.99)])

In [19]:
df = pd.DataFrame([{
    "Codice Prodotto": p.product_code,
    "Descrizione": p.description,
    "Quantità": p.quantity,
    "Prezzo": p.price
} for p in articoli_fattura.products])

display(df)

#### per PDF con dentro scansioni o immagini / diagrammi si possono usare librerie dedicate come ad esempio https://github.com/opendatalab/PDF-Extract-Kit

,Codice Prodotto,Descrizione,Quantità,Prezzo
0,DLRG-23,Faretti LED 40W RGB,1,118.50
1,STRD-213,Asta microfonica Rode PSA1+ per microfoni a co...,1,104.99
2,MCRD-12,Microfono Rode PodMic-USB,1,139.00
3,CVSB-71,Cavo USB-C / USB-A 3 metri schermato,2,9.99
4,WBLG-9,Webcam Logitech Brio 4k con supporto da monitor,1,129.91
5,STDK-3,Elgato Stream Deck 15 tasti,1,119.99


Ovviamente, in entrambi i casi, se passiamo un input che non ha senso rispetto alla richiesta, il modello sarà comunque forzato a rispettare la struttura richiesta a potrebbe dunque portare a risultati insensati o errati

In [20]:
products_llm.invoke("descrivi brevemente lo spazio latente dei modelli di linguaggio")

Products_List(products=[Product(product_code='001', description='Spazio latente nei modelli di linguaggio', quantity='1', price=0.0)])